In [135]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
X_train = np.load("dataset/x_train.npy")
y_train = np.load("dataset/y_train.npy")
X_val = np.load("dataset/x_val.npy")
y_val = np.load("dataset/y_val.npy")
X_test = np.load("dataset/x_test.npy")
y_test = np.load("dataset/y_test.npy")

In [7]:
input_shape = X_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [38]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model_init = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
teacher_model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
student_model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)

In [32]:
def train(model, epochs, train_loader, val_loader, optimizer, loss_function, temperature, patience, patience_counter, min_delta, best_loss):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs/temperature, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
    
        avg_train_loss = train_loss / len(train_loader)
    
        model.eval()
        val_train_loss = 0.0
        correct_predictions = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_function(outputs/temperature, labels)
                val_train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
    
        avg_val_loss = val_train_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_dataset)
    
        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
        # Early stopping check using min_delta
        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
    
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

In [67]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()
temperature = 1

min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

train(model, 50, train_loader, val_loader, optimizer, loss_function, temperature, patience, patience_counter, min_delta, best_loss)

Epoch 1, Training Loss: 0.1565, Validation Loss: 0.1387, Validation Accuracy: 0.9336
Epoch 2, Training Loss: 0.1347, Validation Loss: 0.1348, Validation Accuracy: 0.9354
Epoch 3, Training Loss: 0.1323, Validation Loss: 0.1316, Validation Accuracy: 0.9387
Epoch 4, Training Loss: 0.1275, Validation Loss: 0.1257, Validation Accuracy: 0.9382
Epoch 5, Training Loss: 0.1254, Validation Loss: 0.1259, Validation Accuracy: 0.9382
Epoch 6, Training Loss: 0.1244, Validation Loss: 0.1261, Validation Accuracy: 0.9395
Epoch 7, Training Loss: 0.1193, Validation Loss: 0.1133, Validation Accuracy: 0.9431
Epoch 8, Training Loss: 0.1077, Validation Loss: 0.1277, Validation Accuracy: 0.9431
Epoch 9, Training Loss: 0.1063, Validation Loss: 0.1065, Validation Accuracy: 0.9478
Epoch 10, Training Loss: 0.1050, Validation Loss: 0.1069, Validation Accuracy: 0.9463
Epoch 11, Training Loss: 0.1050, Validation Loss: 0.1058, Validation Accuracy: 0.9469
Epoch 12, Training Loss: 0.1047, Validation Loss: 0.1056, Valid

In [68]:
torch.save(model.state_dict(), 'model/model.pt')

In [34]:
optimizer = optim.Adam(model_init.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()
temperature = 1

min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

train(model_init, 1, train_loader, val_loader, optimizer, loss_function, temperature, patience, patience_counter, min_delta, best_loss)

Epoch 1, Training Loss: 0.1591, Validation Loss: 0.1372, Validation Accuracy: 0.9327


In [35]:
torch.save(model_init.state_dict(), 'model/model_init.pt')

In [39]:
optimizer = optim.Adam(teacher_model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()
temperature = 100

teacher_model.load_state_dict(torch.load('model/model_init.pt'))

min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [40]:
train(teacher_model, 50, train_loader, val_loader, optimizer, loss_function, temperature, patience, patience_counter, min_delta, best_loss)

Epoch 1, Training Loss: 0.1607, Validation Loss: 0.1393, Validation Accuracy: 0.9320
Epoch 2, Training Loss: 0.1362, Validation Loss: 0.1379, Validation Accuracy: 0.9354
Epoch 3, Training Loss: 0.1342, Validation Loss: 0.1349, Validation Accuracy: 0.9361
Epoch 4, Training Loss: 0.1334, Validation Loss: 0.1353, Validation Accuracy: 0.9351
Epoch 5, Training Loss: 0.1328, Validation Loss: 0.1340, Validation Accuracy: 0.9360
Epoch 6, Training Loss: 0.1323, Validation Loss: 0.1342, Validation Accuracy: 0.9338
Epoch 7, Training Loss: 0.1318, Validation Loss: 0.1329, Validation Accuracy: 0.9365
Epoch 8, Training Loss: 0.1312, Validation Loss: 0.1320, Validation Accuracy: 0.9362
Epoch 9, Training Loss: 0.1297, Validation Loss: 0.1339, Validation Accuracy: 0.9333
Epoch 10, Training Loss: 0.1280, Validation Loss: 0.1291, Validation Accuracy: 0.9364
Epoch 11, Training Loss: 0.1264, Validation Loss: 0.1275, Validation Accuracy: 0.9374
Epoch 12, Training Loss: 0.1252, Validation Loss: 0.1267, Valid

In [65]:
torch.save(teacher_model.state_dict(), 'model/teacher_model.pt')

In [149]:
teacher_model.eval()
inputs_list = []
soft_labels = []

with torch.no_grad():
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        outputs = teacher_model(inputs)
        outputs = F.softmax(outputs, dim=1)
        outputs = outputs.detach().cpu()
        inputs_list.append(inputs.cpu())
        soft_labels.append(outputs)

X_train_student = torch.cat(inputs_list)
y_train_student = torch.cat(soft_labels)

In [146]:
y_train_student[1]

tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [155]:
X_train_student = X_train_student.to(device)
y_train_student = y_train_student.to(device)

In [156]:
student_train_dataset = TensorDataset(X_train_student, y_train_student)
student_train_loader = DataLoader(student_train_dataset, batch_size = 100, shuffle = True)

In [157]:
optimizer = optim.Adam(student_model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()
temperature = 100

student_model.load_state_dict(torch.load('model/model_init.pt'))

min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [158]:
train(student_model, 50, student_train_loader, val_loader, optimizer, loss_function, temperature, patience, patience_counter, min_delta, best_loss)

Epoch 1, Training Loss: 0.0737, Validation Loss: 0.3439, Validation Accuracy: 0.9332
Epoch 2, Training Loss: 0.0410, Validation Loss: 0.4550, Validation Accuracy: 0.9355
Epoch 3, Training Loss: 0.0351, Validation Loss: 0.5562, Validation Accuracy: 0.9370
Epoch 4, Training Loss: 0.0334, Validation Loss: 0.6062, Validation Accuracy: 0.9358
Epoch 5, Training Loss: 0.0319, Validation Loss: 0.6454, Validation Accuracy: 0.9365
Epoch 6, Training Loss: 0.0306, Validation Loss: 0.6732, Validation Accuracy: 0.9369
Early stopping triggered


In [159]:
torch.save(student_model.state_dict(), 'model/student_model.pt')

In [87]:
columns = ['Dataset', 'Accuracy', 'Macro Precision', 'Macro Recall', 'Macro F1', 'Macro AUC', 'Weighted Precision', 'Weighted Recall', 'Weighted F1', 'Weighted AUC', 'TPR', 'TNR', 'FPR', 'FNR']
results_df = pd.DataFrame(columns=columns)

In [75]:
def performance(X_test, y_true, model, temperature, results_df, dataset_name, model_name):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            outputs = outputs/temperature
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro', zero_division=True)
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=True)
    
        macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities, average='macro', multi_class='ovr')
        weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities, average='weighted', multi_class='ovr')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        #print(f"Accuracy: {accuracy}")
        
        #print("\nmacro")
        #print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}\nAUC: {macro_auc}")
    
        #print("\nweighted")
        #print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}\nAUC: {weighted_auc}")
        #print()
        
        #print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            'Model': model_name,
            'Dataset': dataset_name,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Macro Recall': recall_macro,
            'Macro F1': f1_macro,
            'Macro AUC': macro_auc,
            'Weighted Precision': precision_weighted,
            'Weighted Recall': recall_weighted,
            'Weighted F1': f1_weighted,
            'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }

        new_row_df = pd.DataFrame([new_row])  # Convert your new row into a DataFrame
        results_df = pd.concat([results_df, new_row_df], ignore_index=True)
        return results_df

In [88]:
results_df = performance(X_test, y_test, model, 1, results_df, 'Edge-IIoT', 'original dnn temp = 1')

C:\Users\Charlie Shang\AppData\Local\Temp\ipykernel_53968\3170879833.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row_df], ignore_index=True)


In [89]:
results_df = performance(X_test, y_test, model, 100, results_df, 'Edge-IIoT', 'original dnn temp = 100')

In [90]:
results_df

,Dataset,Accuracy,Macro Precision,Macro Recall,Macro F1,Macro AUC,Weighted Precision,Weighted Recall,Weighted F1,Weighted AUC,TPR,TNR,FPR,FNR,Model
0,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.996163,0.960395,0.948483,0.944165,0.998802,0.782059,0.996479,0.003521,0.217941,original dnn temp = 1
1,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.885801,0.960395,0.948483,0.944165,0.982763,0.782059,0.996479,0.003521,0.217941,original dnn temp = 100


In [91]:
results_df = performance(X_test, y_test, teacher_model, 1, results_df, 'Edge-IIoT', 'teacher dnn temp = 1')

In [92]:
results_df = performance(X_test, y_test, teacher_model, 100, results_df, 'Edge-IIoT', 'teacher dnn temp = 100')

In [93]:
results_df

,Dataset,Accuracy,Macro Precision,Macro Recall,Macro F1,Macro AUC,Weighted Precision,Weighted Recall,Weighted F1,Weighted AUC,TPR,TNR,FPR,FNR,Model
0,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.996163,0.960395,0.948483,0.944165,0.998802,0.782059,0.996479,0.003521,0.217941,original dnn temp = 1
1,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.885801,0.960395,0.948483,0.944165,0.982763,0.782059,0.996479,0.003521,0.217941,original dnn temp = 100
2,Edge-IIoT,0.948750,0.850622,0.770114,0.778132,0.960629,0.960691,0.948750,0.944285,0.993953,0.770114,0.996497,0.003503,0.229886,teacher dnn temp = 1
3,Edge-IIoT,0.948750,0.850622,0.770114,0.778132,0.996204,0.960691,0.948750,0.944285,0.998818,0.770114,0.996497,0.003503,0.229886,teacher dnn temp = 100


In [160]:
results_df = performance(X_test, y_test, student_model, 1, results_df, 'Edge-IIoT', 'student dnn temp = 1')

In [161]:
results_df = performance(X_test, y_test, student_model, 100, results_df, 'Edge-IIoT', 'student dnn temp = 100')

In [96]:
results_df

,Dataset,Accuracy,Macro Precision,Macro Recall,Macro F1,Macro AUC,Weighted Precision,Weighted Recall,Weighted F1,Weighted AUC,TPR,TNR,FPR,FNR,Model
0,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.996163,0.960395,0.948483,0.944165,0.998802,0.782059,0.996479,0.003521,0.217941,original dnn temp = 1
1,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.885801,0.960395,0.948483,0.944165,0.982763,0.782059,0.996479,0.003521,0.217941,original dnn temp = 100
2,Edge-IIoT,0.948750,0.850622,0.770114,0.778132,0.960629,0.960691,0.948750,0.944285,0.993953,0.770114,0.996497,0.003503,0.229886,teacher dnn temp = 1
3,Edge-IIoT,0.948750,0.850622,0.770114,0.778132,0.996204,0.960691,0.948750,0.944285,0.998818,0.770114,0.996497,0.003503,0.229886,teacher dnn temp = 100
4,Edge-IIoT,0.714517,0.980968,0.066667,0.055566,0.500000,0.796017,0.714517,0.595543,0.500000,0.066667,0.933333,0.066667,0.933333,student dnn temp = 1
5,Edge-IIoT,0.714517,0.980968,0.066667,0.055566,0.670023,0.796017,0.714517,0.595543,0.812657,0.066667,0.933333,0.066667,0.933333,student dnn temp = 100


In [102]:
BIM001 = np.load("dataset/BIM_001.npy")
BIM005 = np.load("dataset/BIM_005.npy")
BIM01 = np.load("dataset/BIM_01.npy")
BIM02 = np.load("dataset/BIM_02.npy")
BIM03 = np.load("dataset/BIM_03.npy")

PGD001 = np.load("dataset/PGD_001.npy")
PGD005 = np.load("dataset/PGD_005.npy")
PGD01 = np.load("dataset/PGD_01.npy")
PGD02 = np.load("dataset/PGD_02.npy")
PGD03 = np.load("dataset/PGD_03.npy")

FGSM001 = np.load("dataset/FGSM_001.npy")
FGSM005 = np.load("dataset/FGSM_005.npy")
FGSM01 = np.load("dataset/FGSM_01.npy")
FGSM02 = np.load("dataset/FGSM_02.npy")
FGSM03 = np.load("dataset/FGSM_03.npy")

DF001 = np.load("dataset/DF_001.npy")
DF005 = np.load("dataset/DF_005.npy")
DF01 = np.load("dataset/DF_01.npy")
DF02 = np.load("dataset/DF_02.npy")
DF03 = np.load("dataset/DF_03.npy")

BA001 = np.load("dataset/BA_001.npy")
BA005 = np.load("dataset/BA_005.npy")
BA01 = np.load("dataset/BA_01.npy")
BA015 = np.load("dataset/BA_015.npy")
BA02 = np.load("dataset/BA_02.npy")
BA025 = np.load("dataset/BA_025.npy")
BA03 = np.load("dataset/BA_03.npy")

ZOO001 = np.load("dataset/ZOO_001.npy")
ZOO005 = np.load("dataset/ZOO_005.npy")
ZOO01 = np.load("dataset/ZOO_01.npy")
ZOO02 = np.load("dataset/ZOO_02.npy")
ZOO03 = np.load("dataset/ZOO_03.npy")

A2PM001 = np.load("dataset/A2PM_001.npy")
# A2PM005 = np.load("dataset/A2PM_005.npy")
A2PM01 = np.load("dataset/A2PM_01.npy")
# A2PM02 = np.load("dataset/A2PM_02.npy")
# A2PM03 = np.load("dataset/A2PM_03.npy")
# A2PM05 = np.load("dataset/A2PM_05.npy")

In [105]:
results_df = performance(BIM001, y_test, model, 1, results_df, 'BIM001', 'original dnn temp = 1')
print()
results_df = performance(BIM005, y_test, model, 1, results_df, 'BIM005', 'original dnn temp = 1')
print()
results_df = performance(BIM01, y_test, model, 1, results_df, 'BIM01', 'original dnn temp = 1')
print()
results_df = performance(BIM02, y_test, model, 1, results_df, 'BIM02', 'original dnn temp = 1')
print()
results_df = performance(BIM03, y_test, model, 1, results_df, 'BIM03', 'original dnn temp = 1')
print()

results_df = performance(BIM001, y_test, model, 100, results_df, 'BIM001', 'original dnn temp = 100')
print()
results_df = performance(BIM005, y_test, model, 100, results_df, 'BIM005', 'original dnn temp = 100')
print()
results_df = performance(BIM01, y_test, model, 100, results_df, 'BIM01', 'original dnn temp = 100')
print()
results_df = performance(BIM02, y_test, model, 100, results_df, 'BIM02', 'original dnn temp = 100')
print()
results_df = performance(BIM03, y_test, model, 100, results_df, 'BIM03', 'original dnn temp = 100')

In [106]:
results_df = performance(PGD001, y_test, model, 1, results_df, 'PGD001', 'original dnn temp = 1')
print()
results_df = performance(PGD005, y_test, model, 1, results_df, 'PGD005', 'original dnn temp = 1')
print()
results_df = performance(PGD01, y_test, model, 1, results_df, 'PGD01', 'original dnn temp = 1')
print()
results_df = performance(PGD02, y_test, model, 1, results_df, 'PGD02', 'original dnn temp = 1')
print()
results_df = performance(PGD03, y_test, model, 1, results_df, 'PGD03', 'original dnn temp = 1')
print()

results_df = performance(PGD001, y_test, model, 100, results_df, 'PGD001', 'original dnn temp = 100')
print()
results_df = performance(PGD005, y_test, model, 100, results_df, 'PGD005', 'original dnn temp = 100')
print()
results_df = performance(PGD01, y_test, model, 100, results_df, 'PGD01', 'original dnn temp = 100')
print()
results_df = performance(PGD02, y_test, model, 100, results_df, 'PGD02', 'original dnn temp = 100')
print()
results_df = performance(PGD03, y_test, model, 100, results_df, 'PGD03', 'original dnn temp = 100')

In [107]:
results_df = performance(FGSM001, y_test, model, 1, results_df, 'FGSM001', 'original dnn temp = 1')
print()
results_df = performance(FGSM005, y_test, model, 1, results_df, 'FGSM005', 'original dnn temp = 1')
print()
results_df = performance(FGSM01, y_test, model, 1, results_df, 'FGSM01', 'original dnn temp = 1')
print()
results_df = performance(FGSM02, y_test, model, 1, results_df, 'FGSM02', 'original dnn temp = 1')
print()
results_df = performance(FGSM03, y_test, model, 1, results_df, 'FGSM03', 'original dnn temp = 1')
print()

results_df = performance(FGSM001, y_test, model, 100, results_df, 'FGSM001', 'original dnn temp = 100')
print()
results_df = performance(FGSM005, y_test, model, 100, results_df, 'FGSM001', 'original dnn temp = 100')
print()
results_df = performance(FGSM01, y_test, model, 100, results_df, 'FGSM001', 'original dnn temp = 100')
print()
results_df = performance(FGSM02, y_test, model, 100, results_df, 'FGSM001', 'original dnn temp = 100')
print()
results_df = performance(FGSM03, y_test, model, 100, results_df, 'FGSM001', 'original dnn temp = 100')

In [108]:
results_df = performance(DF001, y_test, model, 1, results_df, 'DF001', 'original dnn temp = 1')
print()
results_df = performance(DF005, y_test, model, 1, results_df, 'DF005', 'original dnn temp = 1')
print()
results_df = performance(DF01, y_test, model, 1, results_df, 'DF01', 'original dnn temp = 1')
print()
results_df = performance(DF02, y_test, model, 1, results_df, 'DF02', 'original dnn temp = 1')
print()
results_df = performance(DF03, y_test, model, 1, results_df, 'DF03', 'original dnn temp = 1')
print()

results_df = performance(DF001, y_test, model, 100, results_df, 'DF001', 'original dnn temp = 100')
print()
results_df = performance(DF005, y_test, model, 100, results_df, 'DF005', 'original dnn temp = 100')
print()
results_df = performance(DF01, y_test, model, 100, results_df, 'DF01', 'original dnn temp = 100')
print()
results_df = performance(DF02, y_test, model, 100, results_df, 'DF02', 'original dnn temp = 100')
print()
results_df = performance(DF03, y_test, model, 100, results_df, 'DF03', 'original dnn temp = 100')

In [109]:
results_df = performance(BA001, y_test, model, 1, results_df, 'BA001', 'original dnn temp = 1')
print()
results_df = performance(BA005, y_test, model, 1, results_df, 'BA005', 'original dnn temp = 1')
print()
results_df = performance(BA01, y_test, model, 1, results_df, 'BA01', 'original dnn temp = 1')
print()
results_df = performance(BA02, y_test, model, 1, results_df, 'BA02', 'original dnn temp = 1')
print()
results_df = performance(BA03, y_test, model, 1, results_df, 'BA03', 'original dnn temp = 1')

print()
results_df = performance(BA001, y_test, model, 100, results_df, 'BA001', 'original dnn temp = 100')
print()
results_df = performance(BA005, y_test, model, 100, results_df, 'BA005', 'original dnn temp = 100')
print()
results_df = performance(BA01, y_test, model, 100, results_df, 'BA01', 'original dnn temp = 100')
print()
results_df = performance(BA02, y_test, model, 100, results_df, 'BA02', 'original dnn temp = 100')
print()
results_df = performance(BA03, y_test, model, 100, results_df, 'BA03', 'original dnn temp = 100')

In [110]:
results_df = performance(ZOO001, y_test, model, 1, results_df, 'ZOO001', 'original dnn temp = 1')
print()
results_df = performance(ZOO005, y_test, model, 1, results_df, 'ZOO005', 'original dnn temp = 1')
print()
results_df = performance(ZOO01, y_test, model, 1, results_df, 'ZOO01', 'original dnn temp = 1')
print()
results_df = performance(ZOO02, y_test, model, 1, results_df, 'ZOO02', 'original dnn temp = 1')
print()
results_df = performance(ZOO03, y_test, model, 1, results_df, 'ZOO03', 'original dnn temp = 1')
print()

results_df = performance(ZOO001, y_test, model, 100, results_df, 'ZOO001', 'original dnn temp = 100')
print()
results_df = performance(ZOO005, y_test, model, 100, results_df, 'ZOO005', 'original dnn temp = 100')
print()
results_df = performance(ZOO01, y_test, model, 100, results_df, 'ZOO01', 'original dnn temp = 100')
print()
results_df = performance(ZOO02, y_test, model, 100, results_df, 'ZOO02', 'original dnn temp = 100')
print()
results_df = performance(ZOO03, y_test, model, 100, results_df, 'ZOO03', 'original dnn temp = 100')

In [111]:
results_df = performance(A2PM001, y_test, model, 1, results_df, 'A2PM001', 'original dnn temp = 1')
# results_df = performance(A2PM005, y_test, model, 1, results_df, 'A2PM005', 'original dnn temp = 1')
print()
results_df = performance(A2PM01, y_test, model, 1, results_df, 'A2PM01', 'original dnn temp = 1')
# results_df = performance(A2PM02, y_test, model, 1, results_df, 'A2PM02', 'original dnn temp = 1')
# results_df = performance(A2PM03, y_test, model, 1, results_df, 'A2PM03', 'original dnn temp = 1')

results_df = performance(A2PM001, y_test, model, 100, results_df, 'A2PM001', 'original dnn temp = 100')
print()
# results_df = performance(A2PM005, y_test, model, 100, results_df, 'A2PM005', 'original dnn temp = 100')
results_df = performance(A2PM01, y_test, model, 100, results_df, 'A2PM01', 'original dnn temp = 100')
print()
# results_df = performance(A2PM02, y_test, model, 100, results_df, 'A2PM02', 'original dnn temp = 100')
# results_df = performance(A2PM03, y_test, model, 100, results_df, 'A2PM03', 'original dnn temp = 100')

In [112]:
results_df

,Dataset,Accuracy,Macro Precision,Macro Recall,Macro F1,Macro AUC,Weighted Precision,Weighted Recall,Weighted F1,Weighted AUC,TPR,TNR,FPR,FNR,Model
0,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.996163,0.960395,0.948483,0.944165,0.998802,0.782059,0.996479,0.003521,0.217941,original dnn temp = 1
1,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.885801,0.960395,0.948483,0.944165,0.982763,0.782059,0.996479,0.003521,0.217941,original dnn temp = 100
2,Edge-IIoT,0.948750,0.850622,0.770114,0.778132,0.960629,0.960691,0.948750,0.944285,0.993953,0.770114,0.996497,0.003503,0.229886,teacher dnn temp = 1
3,Edge-IIoT,0.948750,0.850622,0.770114,0.778132,0.996204,0.960691,0.948750,0.944285,0.998818,0.770114,0.996497,0.003503,0.229886,teacher dnn temp = 100
4,Edge-IIoT,0.714517,0.980968,0.066667,0.055566,0.500000,0.796017,0.714517,0.595543,0.500000,0.066667,0.933333,0.066667,0.933333,student dnn temp = 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,ZOO03,0.582374,0.265897,0.285672,0.232344,0.761076,0.818203,0.582374,0.670229,0.856194,0.285672,0.971582,0.028418,0.714328,original dnn temp = 100
72,A2PM001,0.877435,0.663158,0.625844,0.600687,0.972111,0.920513,0.877435,0.890724,0.987700,0.625844,0.991673,0.008327,0.374156,original dnn temp = 1
73,A2PM01,0.867514,0.577863,0.596987,0.561853,0.977015,0.912829,0.867514,0.884399,0.984600,0.596987,0.991005,0.008995,0.403013,original dnn temp = 1
74,A2PM001,0.877435,0.663158,0.625844,0.600687,0.874155,0.920513,0.877435,0.890724,0.979553,0.625844,0.991673,0.008327,0.374156,original dnn temp = 100


In [113]:
results_df = performance(BIM001, y_test, teacher_model, 1, results_df, 'BIM001', 'teacher dnn temp = 1')
print()
results_df = performance(BIM005, y_test, teacher_model, 1, results_df, 'BIM005', 'teacher dnn temp = 1')
print()
results_df = performance(BIM01, y_test, teacher_model, 1, results_df, 'BIM01', 'teacher dnn temp = 1')
print()
results_df = performance(BIM02, y_test, teacher_model, 1, results_df, 'BIM02', 'teacher dnn temp = 1')
print()
results_df = performance(BIM03, y_test, teacher_model, 1, results_df, 'BIM03', 'teacher dnn temp = 1')
print()

results_df = performance(BIM001, y_test, teacher_model, 100, results_df, 'BIM001', 'teacher dnn temp = 100')
print()
results_df = performance(BIM005, y_test, teacher_model, 100, results_df, 'BIM005', 'teacher dnn temp = 100')
print()
results_df = performance(BIM01, y_test, teacher_model, 100, results_df, 'BIM01', 'teacher dnn temp = 100')
print()
results_df = performance(BIM02, y_test, teacher_model, 100, results_df, 'BIM02', 'teacher dnn temp = 100')
print()
results_df = performance(BIM03, y_test, teacher_model, 100, results_df, 'BIM03', 'teacher dnn temp = 100')

In [114]:
results_df = performance(PGD001, y_test, teacher_model, 1, results_df, 'PGD001', 'teacher dnn temp = 1')
print()
results_df = performance(PGD005, y_test, teacher_model, 1, results_df, 'PGD005', 'teacher dnn temp = 1')
print()
results_df = performance(PGD01, y_test, teacher_model, 1, results_df, 'PGD01', 'teacher dnn temp = 1')
print()
results_df = performance(PGD02, y_test, teacher_model, 1, results_df, 'PGD02', 'teacher dnn temp = 1')
print()
results_df = performance(PGD03, y_test, teacher_model, 1, results_df, 'PGD03', 'teacher dnn temp = 1')
print()

results_df = performance(PGD001, y_test, teacher_model, 100, results_df, 'PGD001', 'teacher dnn temp = 100')
print()
results_df = performance(PGD005, y_test, teacher_model, 100, results_df, 'PGD005', 'teacher dnn temp = 100')
print()
results_df = performance(PGD01, y_test, teacher_model, 100, results_df, 'PGD01', 'teacher dnn temp = 100')
print()
results_df = performance(PGD02, y_test, teacher_model, 100, results_df, 'PGD02', 'teacher dnn temp = 100')
print()
results_df = performance(PGD03, y_test, teacher_model, 100, results_df, 'PGD03', 'teacher dnn temp = 100')

In [115]:
results_df = performance(FGSM001, y_test, teacher_model, 1, results_df, 'FGSM001', 'teacher dnn temp = 1')
results_df = performance(FGSM005, y_test, teacher_model, 1, results_df, 'FGSM005', 'teacher dnn temp = 1')
results_df = performance(FGSM01, y_test, teacher_model, 1, results_df, 'FGSM01', 'teacher dnn temp = 1')
results_df = performance(FGSM02, y_test, teacher_model, 1, results_df, 'FGSM02', 'teacher dnn temp = 1')
results_df = performance(FGSM03, y_test, teacher_model, 1, results_df, 'FGSM03', 'teacher dnn temp = 1')

results_df = performance(FGSM001, y_test, teacher_model, 100, results_df, 'FGSM001', 'teacher dnn temp = 100')
results_df = performance(FGSM005, y_test, teacher_model, 100, results_df, 'FGSM001', 'teacher dnn temp = 100')
results_df = performance(FGSM01, y_test, teacher_model, 100, results_df, 'FGSM001', 'teacher dnn temp = 100')
results_df = performance(FGSM02, y_test, teacher_model, 100, results_df, 'FGSM001', 'teacher dnn temp = 100')
results_df = performance(FGSM03, y_test, teacher_model, 100, results_df, 'FGSM001', 'teacher dnn temp = 100')

In [116]:
results_df = performance(DF001, y_test, teacher_model, 1, results_df, 'DF001', 'teacher dnn temp = 1')
results_df = performance(DF005, y_test, teacher_model, 1, results_df, 'DF005', 'teacher dnn temp = 1')
results_df = performance(DF01, y_test, teacher_model, 1, results_df, 'DF01', 'teacher dnn temp = 1')
results_df = performance(DF02, y_test, teacher_model, 1, results_df, 'DF02', 'teacher dnn temp = 1')
results_df = performance(DF03, y_test, teacher_model, 1, results_df, 'DF03', 'teacher dnn temp = 1')

results_df = performance(DF001, y_test, teacher_model, 100, results_df, 'DF001', 'teacher dnn temp = 100')
results_df = performance(DF005, y_test, teacher_model, 100, results_df, 'DF005', 'teacher dnn temp = 100')
results_df = performance(DF01, y_test, teacher_model, 100, results_df, 'DF01', 'teacher dnn temp = 100')
results_df = performance(DF02, y_test, teacher_model, 100, results_df, 'DF02', 'teacher dnn temp = 100')
results_df = performance(DF03, y_test, teacher_model, 100, results_df, 'DF03', 'teacher dnn temp = 100')

In [117]:
results_df = performance(BA001, y_test, teacher_model, 1, results_df, 'BA001', 'teacher dnn temp = 1')
results_df = performance(BA005, y_test, teacher_model, 1, results_df, 'BA005', 'teacher dnn temp = 1')
results_df = performance(BA01, y_test, teacher_model, 1, results_df, 'BA01', 'teacher dnn temp = 1')
results_df = performance(BA02, y_test, teacher_model, 1, results_df, 'BA02', 'teacher dnn temp = 1')
results_df = performance(BA03, y_test, teacher_model, 1, results_df, 'BA03', 'teacher dnn temp = 1')

results_df = performance(BA001, y_test, teacher_model, 100, results_df, 'BA001', 'teacher dnn temp = 100')
results_df = performance(BA005, y_test, teacher_model, 100, results_df, 'BA005', 'teacher dnn temp = 100')
results_df = performance(BA01, y_test, teacher_model, 100, results_df, 'BA01', 'teacher dnn temp = 100')
results_df = performance(BA02, y_test, teacher_model, 100, results_df, 'BA02', 'teacher dnn temp = 100')
results_df = performance(BA03, y_test, teacher_model, 100, results_df, 'BA03', 'teacher dnn temp = 100')

In [118]:
results_df = performance(ZOO001, y_test, teacher_model, 1, results_df, 'ZOO001', 'teacher dnn temp = 1')
results_df = performance(ZOO005, y_test, teacher_model, 1, results_df, 'ZOO005', 'teacher dnn temp = 1')
results_df = performance(ZOO01, y_test, teacher_model, 1, results_df, 'ZOO01', 'teacher dnn temp = 1')
results_df = performance(ZOO02, y_test, teacher_model, 1, results_df, 'ZOO02', 'teacher dnn temp = 1')
results_df = performance(ZOO03, y_test, teacher_model, 1, results_df, 'ZOO03', 'teacher dnn temp = 1')

results_df = performance(ZOO001, y_test, teacher_model, 100, results_df, 'ZOO001', 'teacher dnn temp = 100')
results_df = performance(ZOO005, y_test, teacher_model, 100, results_df, 'ZOO005', 'teacher dnn temp = 100')
results_df = performance(ZOO01, y_test, teacher_model, 100, results_df, 'ZOO01', 'teacher dnn temp = 100')
results_df = performance(ZOO02, y_test, teacher_model, 100, results_df, 'ZOO02', 'teacher dnn temp = 100')
results_df = performance(ZOO03, y_test, teacher_model, 100, results_df, 'ZOO03', 'teacher dnn temp = 100')

In [119]:
results_df = performance(A2PM001, y_test, teacher_model, 1, results_df, 'A2PM001', 'teacher dnn temp = 1')
# results_df = performance(A2PM005, y_test, teacher_model, 1, results_df, 'A2PM005', 'teacher dnn temp = 1')
results_df = performance(A2PM01, y_test, teacher_model, 1, results_df, 'A2PM01', 'teacher dnn temp = 1')
# results_df = performance(A2PM02, y_test, teacher_model, 1, results_df, 'A2PM02', 'teacher dnn temp = 1')
# results_df = performance(A2PM03, y_test, teacher_model, 1, results_df, 'A2PM03', 'teacher dnn temp = 1')

results_df = performance(A2PM001, y_test, teacher_model, 100, results_df, 'A2PM001', 'teacher dnn temp = 100')
# results_df = performance(A2PM005, y_test, teacher_model, 100, results_df, 'A2PM005', 'teacher dnn temp = 100')
results_df = performance(A2PM01, y_test, teacher_model, 100, results_df, 'A2PM01', 'teacher dnn temp = 100')
# results_df = performance(A2PM02, y_test, teacher_model, 100, results_df, 'A2PM02', 'teacher dnn temp = 100')
# results_df = performance(A2PM03, y_test, teacher_model, 100, results_df, 'A2PM03', 'teacher dnn temp = 100')

In [162]:
results_df = performance(BIM001, y_test, student_model, 1, results_df, 'BIM001', 'student dnn temp = 1')
results_df = performance(BIM005, y_test, student_model, 1, results_df, 'BIM005', 'student dnn temp = 1')
results_df = performance(BIM01, y_test, student_model, 1, results_df, 'BIM01', 'student dnn temp = 1')
results_df = performance(BIM02, y_test, student_model, 1, results_df, 'BIM02', 'student dnn temp = 1')
results_df = performance(BIM03, y_test, student_model, 1, results_df, 'BIM03', 'student dnn temp = 1')

results_df = performance(BIM001, y_test, student_model, 100, results_df, 'BIM001', 'student dnn temp = 100')
results_df = performance(BIM005, y_test, student_model, 100, results_df, 'BIM005', 'student dnn temp = 100')
results_df = performance(BIM01, y_test, student_model, 100, results_df, 'BIM01', 'student dnn temp = 100')
results_df = performance(BIM02, y_test, student_model, 100, results_df, 'BIM02', 'student dnn temp = 100')
results_df = performance(BIM03, y_test, student_model, 100, results_df, 'BIM03', 'student dnn temp = 100')

In [163]:
results_df = performance(PGD001, y_test, student_model, 1, results_df, 'PGD001', 'student dnn temp = 1')
results_df = performance(PGD005, y_test, student_model, 1, results_df, 'PGD005', 'student dnn temp = 1')
results_df = performance(PGD01, y_test, student_model, 1, results_df, 'PGD01', 'student dnn temp = 1')
results_df = performance(PGD02, y_test, student_model, 1, results_df, 'PGD02', 'student dnn temp = 1')
results_df = performance(PGD03, y_test, student_model, 1, results_df, 'PGD03', 'student dnn temp = 1')

results_df = performance(PGD001, y_test, student_model, 100, results_df, 'PGD001', 'student dnn temp = 100')
results_df = performance(PGD005, y_test, student_model, 100, results_df, 'PGD005', 'student dnn temp = 100')
results_df = performance(PGD01, y_test, student_model, 100, results_df, 'PGD01', 'student dnn temp = 100')
results_df = performance(PGD02, y_test, student_model, 100, results_df, 'PGD02', 'student dnn temp = 100')
results_df = performance(PGD03, y_test, student_model, 100, results_df, 'PGD03', 'student dnn temp = 100')

In [164]:
results_df = performance(FGSM001, y_test, student_model, 1, results_df, 'FGSM001', 'student dnn temp = 1')
results_df = performance(FGSM005, y_test, student_model, 1, results_df, 'FGSM005', 'student dnn temp = 1')
results_df = performance(FGSM01, y_test, student_model, 1, results_df, 'FGSM01', 'student dnn temp = 1')
results_df = performance(FGSM02, y_test, student_model, 1, results_df, 'FGSM02', 'student dnn temp = 1')
results_df = performance(FGSM03, y_test, student_model, 1, results_df, 'FGSM03', 'student dnn temp = 1')

results_df = performance(FGSM001, y_test, student_model, 100, results_df, 'FGSM001', 'student dnn temp = 100')
results_df = performance(FGSM005, y_test, student_model, 100, results_df, 'FGSM001', 'student dnn temp = 100')
results_df = performance(FGSM01, y_test, student_model, 100, results_df, 'FGSM001', 'student dnn temp = 100')
results_df = performance(FGSM02, y_test, student_model, 100, results_df, 'FGSM001', 'student dnn temp = 100')
results_df = performance(FGSM03, y_test, student_model, 100, results_df, 'FGSM001', 'student dnn temp = 100')

In [165]:
results_df = performance(DF001, y_test, student_model, 1, results_df, 'DF001', 'student dnn temp = 1')
results_df = performance(DF005, y_test, student_model, 1, results_df, 'DF005', 'student dnn temp = 1')
results_df = performance(DF01, y_test, student_model, 1, results_df, 'DF01', 'student dnn temp = 1')
results_df = performance(DF02, y_test, student_model, 1, results_df, 'DF02', 'student dnn temp = 1')
results_df = performance(DF03, y_test, student_model, 1, results_df, 'DF03', 'student dnn temp = 1')

results_df = performance(DF001, y_test, student_model, 100, results_df, 'DF001', 'student dnn temp = 100')
results_df = performance(DF005, y_test, student_model, 100, results_df, 'DF005', 'student dnn temp = 100')
results_df = performance(DF01, y_test, student_model, 100, results_df, 'DF01', 'student dnn temp = 100')
results_df = performance(DF02, y_test, student_model, 100, results_df, 'DF02', 'student dnn temp = 100')
results_df = performance(DF03, y_test, student_model, 100, results_df, 'DF03', 'student dnn temp = 100')

In [166]:
results_df = performance(BA001, y_test, student_model, 1, results_df, 'BA001', 'student dnn temp = 1')
results_df = performance(BA005, y_test, student_model, 1, results_df, 'BA005', 'student dnn temp = 1')
results_df = performance(BA01, y_test, student_model, 1, results_df, 'BA01', 'student dnn temp = 1')
results_df = performance(BA02, y_test, student_model, 1, results_df, 'BA02', 'student dnn temp = 1')
results_df = performance(BA03, y_test, student_model, 1, results_df, 'BA03', 'student dnn temp = 1')

results_df = performance(BA001, y_test, student_model, 100, results_df, 'BA001', 'student dnn temp = 100')
results_df = performance(BA005, y_test, student_model, 100, results_df, 'BA005', 'student dnn temp = 100')
results_df = performance(BA01, y_test, student_model, 100, results_df, 'BA01', 'student dnn temp = 100')
results_df = performance(BA02, y_test, student_model, 100, results_df, 'BA02', 'student dnn temp = 100')
results_df = performance(BA03, y_test, student_model, 100, results_df, 'BA03', 'student dnn temp = 100')

In [167]:
results_df = performance(ZOO001, y_test, student_model, 1, results_df, 'ZOO001', 'student dnn temp = 1')
results_df = performance(ZOO005, y_test, student_model, 1, results_df, 'ZOO005', 'student dnn temp = 1')
results_df = performance(ZOO01, y_test, student_model, 1, results_df, 'ZOO01', 'student dnn temp = 1')
results_df = performance(ZOO02, y_test, student_model, 1, results_df, 'ZOO02', 'student dnn temp = 1')
results_df = performance(ZOO03, y_test, student_model, 1, results_df, 'ZOO03', 'student dnn temp = 1')

results_df = performance(ZOO001, y_test, student_model, 100, results_df, 'ZOO001', 'student dnn temp = 100')
results_df = performance(ZOO005, y_test, student_model, 100, results_df, 'ZOO005', 'student dnn temp = 100')
results_df = performance(ZOO01, y_test, student_model, 100, results_df, 'ZOO01', 'student dnn temp = 100')
results_df = performance(ZOO02, y_test, student_model, 100, results_df, 'ZOO02', 'student dnn temp = 100')
results_df = performance(ZOO03, y_test, student_model, 100, results_df, 'ZOO03', 'student dnn temp = 100')

In [168]:
results_df = performance(A2PM001, y_test, student_model, 1, results_df, 'A2PM001', 'student dnn temp = 1')
# results_df = performance(A2PM005, y_test, student_model, 1, results_df, 'A2PM005', 'student dnn temp = 1')
results_df = performance(A2PM01, y_test, student_model, 1, results_df, 'A2PM01', 'student dnn temp = 1')
# results_df = performance(A2PM02, y_test, student_model, 1, results_df, 'A2PM02', 'student dnn temp = 1')
# results_df = performance(A2PM03, y_test, student_model, 1, results_df, 'A2PM03', 'student dnn temp = 1')

results_df = performance(A2PM001, y_test, student_model, 100, results_df, 'A2PM001', 'student dnn temp = 100')
# results_df = performance(A2PM005, y_test, student_model, 100, results_df, 'A2PM005', 'student dnn temp = 100')
results_df = performance(A2PM01, y_test, student_model, 100, results_df, 'A2PM01', 'student dnn temp = 100')
# results_df = performance(A2PM02, y_test, student_model, 100, results_df, 'A2PM02', 'student dnn temp = 100')
# results_df = performance(A2PM03, y_test, student_model, 100, results_df, 'A2PM03', 'student dnn temp = 100')

In [169]:
results_df

,Dataset,Accuracy,Macro Precision,Macro Recall,Macro F1,Macro AUC,Weighted Precision,Weighted Recall,Weighted F1,Weighted AUC,TPR,TNR,FPR,FNR,Model
0,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.996163,0.960395,0.948483,0.944165,0.998802,0.782059,0.996479,0.003521,0.217941,original dnn temp = 1
1,Edge-IIoT,0.948483,0.841154,0.782059,0.780874,0.885801,0.960395,0.948483,0.944165,0.982763,0.782059,0.996479,0.003521,0.217941,original dnn temp = 100
2,Edge-IIoT,0.948750,0.850622,0.770114,0.778132,0.960629,0.960691,0.948750,0.944285,0.993953,0.770114,0.996497,0.003503,0.229886,teacher dnn temp = 1
3,Edge-IIoT,0.948750,0.850622,0.770114,0.778132,0.996204,0.960691,0.948750,0.944285,0.998818,0.770114,0.996497,0.003503,0.229886,teacher dnn temp = 100
4,Edge-IIoT,0.714517,0.980968,0.066667,0.055566,0.500000,0.796017,0.714517,0.595543,0.500000,0.066667,0.933333,0.066667,0.933333,student dnn temp = 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,ZOO03,0.607601,0.271186,0.286084,0.247381,0.837688,0.813202,0.607601,0.687357,0.898290,0.286084,0.972253,0.027747,0.713916,student dnn temp = 100
266,A2PM001,0.905691,0.602472,0.574151,0.521432,0.814479,0.907414,0.905691,0.898547,0.947156,0.574151,0.992324,0.007676,0.425849,student dnn temp = 1
267,A2PM01,0.908225,0.588353,0.585394,0.511996,0.815244,0.911783,0.908225,0.901205,0.948854,0.585394,0.992454,0.007546,0.414606,student dnn temp = 1
268,A2PM001,0.905691,0.602472,0.574151,0.521432,0.961607,0.907414,0.905691,0.898547,0.989614,0.574151,0.992324,0.007676,0.425849,student dnn temp = 100


In [170]:
results_df.to_csv('results.csv', index=False) 